# AgentOps Basic Monitoring 
This is an example of how to use the AgentOps library for basic Agent monitoring with OpenAI's GPT

In [1]:
from openai import OpenAI
import agentops
import os
from dotenv import load_dotenv

/Users/braelynboynton/Developer/agentops/agentops/agentops/client.py:363: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  if len(self._sessions) is not 0:


ImportError: cannot import name 'Client' from partially initialized module 'agentops' (most likely due to a circular import) (/Users/braelynboynton/Developer/agentops/agentops/agentops/__init__.py)

Next, we'll grab our two API keys. You can use dotenv like below or however else you like to load environment variables

In [ ]:
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY") or "<your_openai_key>"
AGENTOPS_API_KEY = os.getenv("AGENTOPS_API_KEY") or "<your_agentops_key>"

The AgentOps library is designed to be a plug-and-play replacement for the OpenAI Client, maximizing use with minimal install effort.

In [ ]:
openai = OpenAI(api_key=OPENAI_API_KEY)
agentops.init(AGENTOPS_API_KEY)

Now just use OpenAI as you would normally!

## Single Session with ChatCompletion

In [ ]:
message = ({"role": "user", "content": "Hello"},)
openai.chat.completions.create(
    model="gpt-3.5-turbo", messages=message, temperature=0.5, stream=True
)

Make sure to end your session with a `Result` (Success|Fail|Indeterminate) for better tracking

In [ ]:
agentops.end_session("Success")

Now if you check the AgentOps dashboard, you should see information related to this run!

# Events
Additionally, you can track custom events via AgentOps.
Let's start a new session and record some events 

In [ ]:
# Create new session
agentops.start_session()

# Optionally, we can add tags to the session
# ao_client.start_session(['Hello Tracker'])

The easiest way to record actions is through the use of AgentOp's decorators

In [ ]:
from agentops import record_function


@record_function("add numbers")
def add(x, y):
    return x + y


add(2, 4)

We can also manually craft an event exactly the way we want

In [ ]:
from agentops import ActionEvent

message = ({"role": "user", "content": "Hello"},)
response = openai.chat.completions.create(
    model="gpt-3.5-turbo", messages=message, temperature=0.5
)

if "hello" in str(response.choices[0].message.content).lower():
    agentops.record(
        ActionEvent(
            action_type="Agent says hello",
            params=str(message),
            returns=str(response.choices[0].message.content),
        )
    )

In [ ]:
agentops.end_session("Success")